In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [15]:
import pandas as pd
import re
import torch
from sklearn.metrics import roc_auc_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

#Loading dataFiles
train_data = pd.read_csv('/content/drive/MyDrive/DataSets_competition/train_set.csv')
dev_data = pd.read_csv('/content/drive/MyDrive/DataSets_competition/dev_set.csv')
test_data = pd.read_csv('/content/drive/MyDrive/DataSets_competition/test_set.csv')

#Cleaning and normalization
def clean_text(text):
    text = re.sub(r"[^\w\s]", " ", text)# Remove punctuation
    text = re.sub(r"\d+", " ", text)# Remove numbers
    text = re.sub(r"[إأآا]", "ا", text)# Normalize alef
    text = re.sub(r"ى", "ي", text)# Normalize ya
    text = re.sub(r"ؤ", "و", text)# Normalize waw
    text = re.sub(r"ء", "", text)#Remove hamza
    text = re.sub(r"ئ", "ي", text)#Normalize ya with hamza
    text = re.sub(r"\s+", " ", text).strip()#Remove extra spaces
    return text

#Applying cleaning
train_data["text"] = train_data["text"].apply(clean_text)
dev_data["text"] = dev_data["text"].apply(clean_text)
test_data["text"] = test_data["text"].apply(clean_text)

#Converting datasets to Hugging Face format
train_dataset = Dataset.from_pandas(train_data[["text", "generated"]])
dev_dataset = Dataset.from_pandas(dev_data[["text", "generated"]])
test_dataset = Dataset.from_pandas(test_data[["text"]])

#Loading AraBERT tokenizer and model
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)

#Ensure num_labels=2 for binary classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

#Tokenization function
def tokenize(batch):
    #Tokenize the text
    encoding = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)
    if "generated" in batch:
        encoding["labels"] = batch["generated"]
    return encoding

#Apply tokenization
train_dataset = train_dataset.map(tokenize, batched=True)
dev_dataset = dev_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)  # No labels for the test set

#Set format for PyTorch tensors
train_dataset = train_dataset.with_format("torch")
dev_dataset = dev_dataset.with_format("torch")

#Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=1,
    metric_for_best_model="eval_roc_auc",
    greater_is_better=True,
    load_best_model_at_end=True,
    seed=42,
    report_to="none"
)

#Defining metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    pred_binary = (predictions >= 0.5).astype(int)
    roc_auc = roc_auc_score(labels, predictions)
    accuracy = accuracy_score(labels, pred_binary)
    return {"roc_auc": roc_auc, "accuracy": accuracy}

#Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#Training the model
trainer.train()

#Evaluating on dev set
eval_results = trainer.evaluate()
print(f"Dev ROC-AUC: {eval_results['eval_roc_auc']:.4f}")
print(f"Dev Accuracy: {eval_results['eval_accuracy']:.4f}")

#Predicting on test set
predictions = trainer.predict(test_dataset)
test_preds = torch.softmax(torch.tensor(predictions.predictions), dim=1)[:, 1].numpy()

#Prepare submission
submission = pd.DataFrame({
    "text_id": test_data["text_id"],
    "generated": test_preds
})
submission.to_csv("submission.csv", index=False)
print("Submission saved as 'submission.csv'")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6763 [00:00<?, ? examples/s]

Map:   0%|          | 0/1450 [00:00<?, ? examples/s]

Map:   0%|          | 0/1451 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-a4efd6bf53d5>:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Roc Auc,Accuracy
1,0.044200,0.019965,0.999802,0.995862
2,0.007400,0.027028,0.999970,0.993793
3,0.005600,0.022828,0.999935,0.994483


Dev ROC-AUC: 1.0000
Dev Accuracy: 0.9938
Submission saved as 'submission.csv'
